In [ ]:
%matplotlib inline
import sys
import os
REPO_PATH = os.getenv('REPO_PATH')
sys.path.insert(0, os.path.join(REPO_PATH, "py_notebooks/"))

import pandas as pd
import numpy as np
import json
from subprocess import call

import myunits
u = myunits.units()

import matplotlib.pyplot as plt
import plotting
import plotly
import plotly.graph_objs as go 
plotly.offline.init_notebook_mode()
from plotly.colors import DEFAULT_PLOTLY_COLORS
import plotly.io as pio

cm_to_in = 0.393701
single_col = 8.3
double_col = 17.1
GRAPH_CONFIG = {'showLink': False, 'displaylogo': False,                         
                'modeBarButtonsToRemove':['sendDataToCloud']}

SAVEFIGS = True

# Previous version

# New figure 6 after R1

In [ ]:
def readdataR1(dc_ratio):
    folderNm = os.path.join(REPO_PATH,
                            "dataR1/jl_out/demand_%.1f"%dc_ratio)
    with open(folderNm+"ne_carbon_json_data") as data_file:    
        data = json.load(data_file)
    res = data[0]
    prices = data[1]
    costs = data[2]
    red = data[3]

    for lb in costs:
        costs[lb] = [0. if c is None else c for c in costs[lb]]
        
    baseC = res['jlInput_x0.csv']['0.0']['CEP_carbon']

    red2 = dict()
    for k in res:
        red2[k] = []
        for price in [0., 50., 75., 100.,150., 200., 300., 500., 1000.,
                      10000., 100000.]:
            red2[k] += [100*(baseC
                             -res[k][str(price)]['CEP_carbon'])/baseC]
    return (red2, costs)


In [ ]:
labs = {'jlInput_x0.csv':"2016", 'jlInput_2018.csv':"2018",'jlInput_x1.csv':"2X solar 2016",
        'jlInput_x2.csv':"2025 (3X solar 2016)", 'jlInput_x4.csv':"5X solar 2016"}
dashes = ['solid', 'dash', 'dot']
opac = [1.0, 0.6, 0.2]
showLeg = [True, False, False]
plot_data = []
for ii,lb in enumerate(labs):
    for idc, dc_ratio in enumerate([1.0, 0.5, 0.1]):
        (red2, costs) = readdataR1(dc_ratio)
        plot_data += [go.Scatter(
            x=red2[lb][:-1],
            y=[c for c in costs[lb][:-1]],
            showlegend=False,
            opacity=opac[idc],
            line = dict(dash=dashes[idc],
                        color = DEFAULT_PLOTLY_COLORS[ii])
                    )]

for idc, dc_ratio in enumerate([1.0, 0.5, 0.1]):
    plot_data += [
        go.Scatter(
            name = "%.1f DC"%dc_ratio,
            x=[-1],
            y=[-100],
            line = dict(dash=dashes[idc],
                        color = "gray")
                        )]

layout = dict(
    font=dict(size=18),
    autosize=False,
    width=650*1.5,
    height=500,
    title='CO2 abatement cost curves for Stanford heating and cooling emissions',
    xaxis=dict(title='Emissions reduction from 2016 BAU levels (%)'),
    yaxis=dict(title='$/tonne', range=[-10., 450.]),
    legend=dict(x=0.8,y=1),
    annotations=[
        dict(x=3,y=300, xref='x', yref='y', text='2016',
             showarrow=False, bgcolor='white'),
        dict(x=13,y=250, xref='x', yref='y', text='2018',
             showarrow=False, bgcolor='white'),
        dict(x=25,y=200, xref='x', yref='y', text='2X solar',
             showarrow=False, bgcolor='white'),
        dict(x=38,y=160, xref='x', yref='y', text='2025 (3X solar)',
             showarrow=False, bgcolor='white'),
        dict(x=50,y=110, xref='x', yref='y', text='5X solar',
             showarrow=False, bgcolor='white')]
)
fig = go.Figure(data=plot_data, layout=layout)
plotly.offline.iplot(fig, config=GRAPH_CONFIG)
#pio.write_image(fig, 'test.pdf')

In [ ]:
if SAVEFIGS:
    call(['xvfb-run', 'orca', 'graph', json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder),
          '-o', 'figures/fig6.pdf', '--format', 'pdf'])